# IVR8 Intermediate Example
This example aims to highlight the follow:
 * demonstrate how to use compartment constraints on a particular model
 * Use a subset of the publist stops and configure a single vehicle
 * Use a simple two-rack compartment configuration to illustrate the basic assignment workings
 * Add a group-limit constraint which only permits loads on the top-rack if there is a task filling the space beneth it.

## Requirements
This example assumes you've configured an api-key with the required services enabled (see the [portal](portal.icepack.ai) for configuration details) and that you're familiar with loading and working with protobuf models.

## Data
We're going to load sample data which has order sizes and durations in a tabular format. The first 9 items will be sufficient for this exercise (item 1 being the depot, and 8 for orders)


In [ ]:
import pandas
df = pandas.read_csv('../sample_data/publist_orders.csv').head(9)
print(df.head())

## Model helper
As with the previous example, we'll be using the model helper and creating a top rack / lower rack style of compartments.

We're going to start by creating an ivr8 model and populating it with some basic information.

In [ ]:
exec(open('apiHelper.py').read()) # import some api-helper classes which we've written for you.
exec(open('ivr8-model-helper.py').read()) # import some modelling helpers

api = apiHelper(modelType="ivr8-yni1c9k2swof") # set the model type to the ivr8 model

sr = ivr8_yni1c9k2swof_pb2.SolveRequest()
sr.solveType = 0 # optimise solve request

m = sr.model
m.dimensions.CopyFrom(make_distance_time_cap_dims())
m.locations.extend(make_locations(df)) 
m.locations.append(m.locations[0]) # just copy the Guiness storehouse location
m.locations[len(m.locations) - 1].id = 'vehicle-site' #rename it and we're good to go (location id's need to be unique in a model)
m.jobs.extend(make_job_time_cap(df, [0] * (df.shape[0]-1), range(1, df.shape[0])))
m.vehicleCostClasses.append(make_vcc_simple('vcc1', 1000, 1.001e-2, 1.001e-2, 1.001e-2, 1, 3))
m.vehicleClasses.append(make_vc_simple('vc1', 1, 1, 1, 1))
m.vehicles.append(make_vehicle_cap('vehicle_1', 'vc1', 'vcc1', #create just one vehicle
                                    2000, # the vehicle capacity
                                    df['id'][0], # the start location
                                    df['id'][0], # the end location
                                    7*60,        # the start time (7AM)
                                    18*60))      # the end time  (6PM)



### Compartment definition
A compartment is a container in which tasks can be placed. Several compartments can be assigned to a vehicle, comprising what we call a `compartment-set`. We can place constraints on individual compartments from an assignmnet perspective (i.e. some tasks may only be assigned to certain compartments) as well as individual constraints on the capacity of a compartment (with respect to some defined dimensions). 

Lets make a simple example where we have a vehicle who's compartments are layed out as follows:


| Rack        |      |   |    |   |
| ------------- |-------------|-----|-----|-----|
| Top Rack      | c1: 100 | c2: 100 |c3: 100 |c4: 100 |
| Bottom Rack   | c5:400      |   c6:400 |c7:400 |c8:400 |

So we have 4 compartments on the top rack (c1-c4) each with a capacity of 100 units, and 4 compartments on the bottom rack each with a holding capacity of 400 units. Lets write the code to build this model!


Compartments can be defined separately from compartments sets.This means we can use any combination of compartment defintions in a compartment set (to define a collection against which constraint checks are performed). We'll start with the compartments

In [ ]:
del m.compartments[:] # in case you run this twice
for i in range(0,8):
    comp = ivr8_yni1c9k2swof_pb2.Compartment()
    comp.id = "c" + str(i+1)
    compcap = ivr8_yni1c9k2swof_pb2.Compartment.Capacity()
    compcap.dimensionId = 'capacity'
    if i < 4:
        compcap.capacity = 100 # top rack
    else:
        compcap.capacity = 400 # bottom rack
    comp.capacities.append(compcap)
    m.compartments.append(comp)
print(m.compartments)

### Compartment set
Now that we have some compartments defined in the model, we can pull them together into a collection (which is then assigned to a vehicle). 

In [ ]:
del m.compartmentSets[:] # in case you run this twice
cset = ivr8_yni1c9k2swof_pb2.CompartmentSet()
cset.id = 'double-decker'
for i in range(0,8):
    cset.compartmentIds.append('c' + str(i + 1)) #indicates that this compartment set has c1:c8 available.
m.compartmentSets.append(cset) # add it to the model

print('Compartment set:')
print(m.compartmentSets)
print('\n\n')
# The last step is to tell a vehicle that it has a compartment set which should be used.
m.vehicles[0].compartmentSetId = 'double-decker'
print(m.vehicles[0])

## Group Limits
In this example we're going to add group limits to the compartments.
Lets say we have a constraint that says something like, "the mass on the top may not exceed the mass on the bottom". This translates to the sum of c1:c4 should be less than the sum of the bottom rack c5:c8.

If we write this out we would say c1 + c2 + c3 + c4 < c5 + c6 + c7 + c8

Which can be rewritten as: c1 + c2 + c3 + c4 - c5 - c6 - c7 - c8 < 0 


In [ ]:
g = ivr8_yni1c9k2swof_pb2.CompartmentSet.GroupLimit()
for i in range(0,8):
    g.compartmentIds.append('c' + str(i + 1))
    if i < 4:
        g.coefficients.append(+1) # this is the left hand side
    else:
        g.coefficients.append(-1) # this is the left hand side
g.dimensionId = 'capacity'
g.limit = 0                       # this is the right hand side
print(g)
del m.compartmentSets[0].groupLimits[:] # in case you run this twice
m.compartmentSets[0].groupLimits.append(g) # add this to the compartment set

## Run the model

Now we can follow the normal process of submitting the model to the api and running it!


In [ ]:
reqId = api.Post(sr)
sol = api.Get(reqId)
t = tabulate(sr, sol)


### Reading the response
We can see in the tabulated response that there is now a `compartmentId` field which gives the allocation of tasks to compartments in the solution. This view of the response is useful, but we can check another view we've coded up to understand now the vehicle behaves over the course of the route.

Lets have a look at the `compartmentSummary` table:

In [ ]:
csummary = t['compartmentSummary']

csummary[list(csummary.keys())[0]] # just grab the compartment summary for the first vehicle.


So this is quite different to the solution in the basic example. 

In this table you can see that the the sum for compartments 1:4 is always less than 5:8. This way we're constrained by always having more weight on the bottom rack than the top rack throughout the route (which is still pretty well costed).

The compartment summary table is a dictionary for each vehicle-capacitated-dimension combination.


### What next?
If this example makes sense you can jump onto the ivr8 advanced example!